In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
from fake_useragent import UserAgent
from datetime import datetime
import time


In [2]:
ua = UserAgent()
headers = {"User-Agent":ua.random}
url = "https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date=2020-05-30&now=Y&filter_live=Y"
res = requests.get(url,headers=headers)
smtv_schedule = json.loads(res.text)



In [3]:
result = []

for i in range(len(smtv_schedule["result"])):
    
    for j in range(len(smtv_schedule["result"][i]["data"])):
        
        smtv_type = smtv_schedule["result"][i]["data"][j]["live_types_name"]
        smtv_type_child = smtv_schedule["result"][i]["data"][j]["live_types_child_name"]
        smtv_lnameEN = smtv_schedule["result"][i]["data"][j]["lname"]
        smtv_time = smtv_schedule["result"][i]["data"][j]["time"]
        smtv_TeamA = smtv_schedule["result"][i]["data"][j]["hname"]
        smtv_TeamB = smtv_schedule["result"][i]["data"][j]["aname"]
        smtv_signal = []

        for k in smtv_schedule["result"][i]["data"][j]["account"]:
            smtv_signal.append(k["nickname"])

        dict_={
            "smtv_type":smtv_type,
            "smtv_type_child":smtv_type_child,
            "smtv_lnameEN":smtv_lnameEN,
            "smtv_time":smtv_time,
            "smtv_TeamA":smtv_TeamA,
            "smtv_TeamB":smtv_TeamB,
            "smtv_signal":smtv_signal
        }
        result.append(dict_)
        
    

In [10]:
len(result)

209

In [11]:
smtv_schedule["result"][]

4

In [5]:
# 平台 賽程時間 賽事 type A隊伍 B隊伍 信號[]

In [6]:
# smtv 體育
# https://smtv.io/program/43
# smtv 娛樂
# https://smtv.io/program/44
# smtv 電競
# https://smtv.io/program/81

#------------------------------

# smzb 體育
# https://smzb.io/program/1
# smzb 娛樂
# https://smzb.io/program/2
# smzb 電競    
# https://smzb.io/program/45

#------------------------------

# ggzb 電競
# https://ggzb.io/

#------------------------------

# lj88 體育
# https://lj88.tv/

#------------------------------

# hqzb 體育
# https://hqzb.tv/program

#------------------------------